In [ ]:
import pandas as pd
import numpy as np

import datetime
print (str(datetime.datetime.today() ) )

print ("Import packages")
#pip install cx_Oracle
#pip install sqlalcheny
#Download Oracle- https://www.oracle.com/database/technologies/instant-client/downloads.html
#Connect to cx_Oracle and import packages
import cx_Oracle

#create path to downloaded oracle files and initiate
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_9")

import sqlalchemy
from sqlalchemy.schema import CreateIndex
from sqlalchemy import Table, Column, Integer, Numeric,String, ForeignKey
from sqlalchemy import MetaData
from sqlalchemy import insert
metadata= MetaData()

from sqlalchemy.engine import create_engine


print ("DONE")

### Import Labor Transactions

### 2023

In [ ]:
path = ""


import pandas as pd

print ("Import labor transactions")

import time
start = time.time()

import datetime
print (str(datetime.datetime.today()).split()[0] )


#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


labtrans_query = """

select labtrans.LABORCODE,
       labtrans.ENTERDATE,
       labtrans.REFWO,
       labtrans.TRANSTYPE,
       labtrans.startdatetime,
       labtrans.finishdatetime,
       labtrans.location

       
from Maximo.labtrans labtrans
--WHERE (labtrans.startdatetime >= '01-JAN-2023' AND labtrans.startdatetime <= '1-SEP-2023') AND labtrans.transtype = 'INSPECTION'
--WHERE (labtrans.startdatetime >= '01-JAN-2023' AND labtrans.startdatetime <= '1-SEP-2023') AND (labtrans.transtype = 'INSPECTION' OR labtrans.transtype = 'NOBLDGACCESS' OR labtrans.transtype = 'RESREFUSED' OR labtrans.transtype ='RESNOTHOME')
WHERE (labtrans.startdatetime >= '01-JAN-2022' AND labtrans.startdatetime <= '31-DEC-2022') 

--WHERE (labtrans.startdatetime >= '01-JAN-2022' AND labtrans.startdatetime <= '31-DEC-2022') AND (labtrans.transtype = 'WORKWITHSEQ' OR labtrans.transtype = 'WORK')
--WHERE TRUNC(labtrans.enterdate) >= sysdate - interval '1' year  AND (labtrans.transtype = 'WORKWITHSEQ' OR labtrans.transtype = 'WORK' OR labtrans.transtype = 'NOBLDGACCESS' OR labtrans.transtype = 'RESREFUSED' OR labtrans.transtype ='RESNOTHOME' OR labtrans.transtype ='UNFOUNDED' )

"""
#(labtrans.enterdate >= '01-JAN-2022')

#WHERE TRUNC(xrf_det.entrydate) >= sysdate - interval '1' year AND xrf_det.finalclassification = 'Positive' AND xrf_det.side != 'Calibrate' AND xrf_det.side != 'CALIBRATE'
#(labtrans.enterdate >= '12-JAN-2023')
#(labtrans.enterdate >= '01-JAN-2022' AND labtrans.enterdate <= '1-MAR-2022')



dl = pd.read_sql_query(labtrans_query, engine)

print (len(dl))

print ("DONE")

In [ ]:
print ("Reading labor transactions...")
path = ""

dl1 = pd.read_csv(path + "Labor Transactions 2022.csv")
dl2 = pd.read_csv(path + "Labor Transactions 2023.csv")

frames = [dl1, dl2]

dl = pd.concat(frames)
dl = dl.reset_index(drop=True)

print (len(dl))

print ("DONE")

In [ ]:
dl.to_csv(path + "Labor Transactions 2022.csv", index=False)

### Import Work Orders to get wonum to subset labor trans and Insert Other Data

### 2022

In [ ]:

import datetime
print (str(datetime.datetime.today()) )


print ("Import work orders")
import time
start = time.time()


DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


workorder_query = """

select max_wo.worktype,
       max_wo.problemcode,
       max_wo.failurecode,
       max_wo.wonum,
       max_wo.parent,
       TRUNC(max_wo.reportdate) AS report_date,
       max_wo.status,
       TRUNC(max_wo.statusdate) AS status_date,
       max_wo.location,
       max_wo.haschildren,
       max_wo.OWNERGROUP,
       max_wo.ACTSTART,
       max_wo.ACTFINISH,
       max_wo.ZZCRAFT,
       max_wo.JPNUM
       
from Maximo.workorder max_wo

--WHERE TRUNC(max_wo.reportdate) >= sysdate - interval '1' year AND max_wo.status = 'CLOSE' AND max_wo.failurecode = 'EXTERMINATION' AND max_wo.worktype = 'CM' 

WHERE (max_wo.actstart >= '01-JAN-2023' AND max_wo.actstart <= '01-SEP-2023') AND max_wo.status = 'CLOSE' AND max_wo.zzcraft = 'MAINT' AND max_wo.jpnum = 'INSAPT'
--WHERE (max_wo.reportdate >= '01-NOV-2021' AND max_wo.reportdate <= '30-NOV-2021') AND max_wo.status = 'CLOSE' AND max_wo.zzcraft = 'MAINT' AND max_wo.jpnum = 'INSAPT'
--WHERE (max_wo.reportdate >= '01-NOV-2021' AND max_wo.reportdate <= '30-NOV-2021') AND max_wo.jpnum = 'INSAPT'


"""




#(max_wo.reportdate >= '01-JAN-2022')
#(max_wo.reportdate >= '01-JAN-2022' AND max_wo.reportdate <= '1-MAR-2022')

df = pd.read_sql_query(workorder_query, engine)

#print (len(df))
#df = df[pd.notnull(df['actstart'])]
#print (len(df))


#print ("Subset for closed work orders")
#print (len(df))
#df = df[df["status"] == "CLOSE"]
#df = df[df["status"] == "APPR"]

#subsetlist = ["CLOSE", "APPR"]

#df = df[df['status'].isin(subsetlist)].reset_index(drop=True)

#print (len(df))
#df = df[df["jpnum"] == 'M10353']
#df = df[df["zzcraft"] == 'EXTERMIN']
#print (len(df))

print ("DONE")

#print (test_df)



In [ ]:
print ("Reading work orders...")
df1 = pd.read_csv(path + "Work Orders 2022.csv")
df2 = pd.read_csv(path + "Work Orders 2023.csv")

frames = [df1, df2]

df = pd.concat(frames)
df = df.reset_index(drop=True)

print (len(df))

print ("DONE")

In [ ]:
#df.to_csv(path + "Work Orders 2023.csv", index=False)

In [ ]:
#df2 = df[df["jpnum"] == 'INSAPT']
#print (len(df))
#print (len(df2))

In [ ]:
#df["jpnum"] = df["jpnum"].astype(str)
#jpnum = sorted(list(set(df["jpnum"].tolist() )))
#jpnum

Subset Labor Transactions

In [ ]:
df["wonum"] = df["wonum"].astype(str).map(str.strip)
dl['refwo'] = dl['refwo'].astype(str).map(str.strip)

dflist = df["wonum"].tolist()

dllist = dl['refwo'].tolist()

x = list(set(dflist).intersection(dllist))

print (len(x))


In [ ]:
import datetime
print (str(datetime.datetime.today()) )

print ("Subset Labor Transactions")

df["wonum"] = df["wonum"].astype(str).map(str.strip)
dl['refwo'] = dl['refwo'].astype(str).map(str.strip)

df = df.reset_index(drop=True)
dflist = df["wonum"].tolist()

dllist = dl['refwo'].tolist()

x = list(set(dflist).intersection(dllist))

print (len(dl))

dl2 = dl[dl['refwo'].isin(dflist)].reset_index(drop=True)

print (len(dl2))

print ("DONE")



In [ ]:
transtype = set(dl2["transtype"].tolist() )

transtype

### Insert Work Order Fields into Labor Transactions

In [ ]:
print (str(datetime.datetime.today()) )


dftypes = ["ownergroup", "zzcraft"]

for d in dftypes:
    dfDic = df.set_index('wonum')[d].to_dict()
    dl2[d] = dl2["refwo"].map(dfDic)

print ("DONE")

### Import Person Data and Merge with Labor Transactions

In [ ]:
print ("Import person data")
print (str(datetime.datetime.today()) )


#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


person_query = """

select *  
       
from Maximo.person 

"""

#WHERE TRUNC(xrf_det.entrydate) >= sysdate - interval '1' year AND xrf_det.finalclassification = 'Positive' AND xrf_det.side != 'Calibrate' AND xrf_det.side != 'CALIBRATE'


dp = pd.read_sql_query(person_query, engine)
dp = dp[["personid", "status", "displayname","firstname","lastname","department","title", "zzhrtitle", "hiredate", "terminationdate"]]
dp["personid"] = dp["personid"].astype(str).map(str.strip)
dp = dp.rename(columns={'personid': 'laborcode'})

print ("DONE")

import datetime
print (str(datetime.datetime.today()) )


dl2["laborcode"] = dl2["laborcode"].astype(str).map(str.strip)
print (len(dl2))
dl2 = pd.merge(dl2, dp, how='left', on=['laborcode'])
print (len(dl2))

print ("DONE")

In [ ]:
#dl2.to_csv(path + "test.csv", index=False)

In [ ]:
#people = sorted(list(set(dl2["displayname"].tolist() ) ))
#print (len(people))
#dp2 = dp[dp['displayname'].isin(people)]



In [ ]:
#dp2.to_csv(path + "Exterminators from Person Dataset.csv", index=False)

In [ ]:
transtype = set(dl2["transtype"].tolist() )
transtype

### Get IPM Inspection Questions

### Import Questions Only  2022

In [ ]:
path = ""


# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)



import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd



ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, wt.actstart,
wt.jpnum, lt.craft,
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  (lt.startdatetime >= '01-SEP-2023' AND lt.startdatetime <= '30-SEP-2023')

and wt.status = 'CLOSE'
and lt.transtype = 'INSPECTION'
and lt.craft = 'MAINT'
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid


"""

#WHERE (max_wo.actstart >= '01-JAN-2022' AND max_wo.actstart <= '31-DEC-2022') AND max_wo.status = 'CLOSE' 
#AND max_wo.zzcraft = 'MAINT' AND max_wo.jpnum = 'INSAPT'


#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
dq = pd.read_sql_query(ques_query, engine)

print (len(dq))

#path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
#dq.to_csv(path + "Labor_Transactions_jan-mar_2022.csv", index=False)

print ("Export")
dq.to_csv(path + "Questions Sep 2023.csv")


print ("DONE")

In [ ]:
"""
jpnum = sorted(list(set(dq["jpnum"].tolist() )))
print (jpnum)

wonum1 = sorted(list(set(dq["wonum1"].tolist() )))
print ("wonum1", len(wonum1))

wonum2 = sorted(list(set(dq["wonum2"].tolist() )))
print ("wonum2", len(wonum2))


transtype = sorted(list(set(dq["transtype"].tolist() )))
print (len(transtype))


craft = sorted(list(set(dq["craft"].tolist() )))
print (len(craft))


description3 = sorted(list(set(dq2["description3"].tolist() )))
print (len(description3))
description3

"""

### Concatinate Data

Import Questions

In [ ]:
dques = pd.read_csv(path + "Questions Prelim.csv", encoding = "utf-8")
desc = sorted(list(set(dques["Description"].tolist() )))

dques["Description"] = dques["Description"].astype(str).map(str.strip).map(str.upper)
dques["Question"] = dques["Question"].astype(str).map(str.strip).map(str.upper)

dques["Question Location"] = dques["Question"] + " " + dques["Description"]

dquesList = sorted(list(set(dques["Question Location"].tolist() )))

dquesList

In [ ]:
pd.set_option('chained_assignment', None)

print (str(datetime.datetime.today()) )

dl2refwoList = sorted(list(set(dl2["refwo"].tolist() )))
print (len(dl2refwoList))

def DataClean(dq,dl2refwoList, dquesList ):
    print (len(dq))
    dq["wonum1"] = dq["wonum1"].astype(str).map(str.strip)
    dq = dq[dq['wonum1'].isin(dl2refwoList)]
    print (len(dq))

    dq["description2"] = dq["description2"].astype(str).map(str.strip).map(str.upper)
    dq["description3"] = dq["description3"].astype(str).map(str.strip).map(str.upper)

    dq["Question Location"] = dq["description3"] + " " + dq["description2"]

    dq = dq[dq['Question Location'].isin(dquesList)].reset_index(drop=True)

    print (len(dq))
    
    return dq

pathq = "C:\\Users\\MehriD01\\Documents\\Apartment Inspections\\Question Data\\"

print ()

dq1 = pd.read_csv(pathq + "Questions Jan 2022.csv", low_memory=False)
print ("Jan 2022")
dq1 = DataClean(dq1,dl2refwoList, dquesList )
print ()

dq2 = pd.read_csv(pathq + "Questions Feb 2022.csv", low_memory=False)
print ("Feb 2022")
dq2 = DataClean(dq2,dl2refwoList, dquesList )
print ()

dq3 = pd.read_csv(pathq + "Questions March 2022.csv", low_memory=False)
print ("March 2022")
dq3 = DataClean(dq3,dl2refwoList, dquesList )
print ()

dq4 = pd.read_csv(pathq + "Questions April 2022.csv",low_memory=False )
print ("April 2022")
dq4 = DataClean(dq4,dl2refwoList, dquesList )
print ()

dq5 = pd.read_csv(pathq + "Questions May 2022.csv", low_memory=False)
print ("May 2022")
dq5 = DataClean(dq5,dl2refwoList, dquesList )
print ()

dq6 = pd.read_csv(pathq + "Questions June 2022.csv", low_memory=False)
print ("June 2022")
dq6 = DataClean(dq6,dl2refwoList, dquesList )
print ()

dq7 = pd.read_csv(pathq + "Questions July 2022.csv", low_memory=False)
print ("July 2022")
dq7 = DataClean(dq7,dl2refwoList, dquesList )
print ()

dq8 = pd.read_csv(pathq + "Questions August 2022.csv", low_memory=False)
print ("Aug 2022")
dq8 = DataClean(dq8,dl2refwoList, dquesList )
print ()

dq9 = pd.read_csv(pathq + "Questions Sept 2022.csv", low_memory=False)
print ("Sept 2022")
dq9 = DataClean(dq9,dl2refwoList, dquesList )
print ()

dq10 = pd.read_csv(pathq + "Questions Oct 2022.csv", low_memory=False)
print ("Oct 2022")
dq10 = DataClean(dq10,dl2refwoList, dquesList )
print ()

dq11 = pd.read_csv(pathq + "Questions Nov 2022.csv", low_memory=False)
print ("Nov 2022")
dq11 = DataClean(dq11,dl2refwoList, dquesList )
print ()

dq12 = pd.read_csv(pathq + "Questions Dec 2022.csv", low_memory=False)
print ("Dec 2022")
dq12 = DataClean(dq12,dl2refwoList, dquesList )
print ()


dq13 = pd.read_csv(pathq + "Questions Jan 2023.csv", low_memory=False)
print ("Jan 2023")
dq13 = DataClean(dq13, dl2refwoList, dquesList )
print ()

dq14 = pd.read_csv(pathq + "Questions Feb 2023.csv", low_memory=False)
print ("Feb 2023")
dq14 = DataClean(dq14, dl2refwoList, dquesList )
print ()

dq15 = pd.read_csv(pathq + "Questions Mar 2023.csv", low_memory=False)
print ("Mar 2023")
dq15 = DataClean(dq15, dl2refwoList, dquesList )
print ()

dq16 = pd.read_csv(pathq + "Questions Apr 2023.csv", low_memory=False)
print ("Apr 2023")
dq16 = DataClean(dq16, dl2refwoList, dquesList )
print ()

dq17 = pd.read_csv(pathq + "Questions May 2023.csv", low_memory=False)
print ("May 2023")
dq17 = DataClean(dq17, dl2refwoList, dquesList )
print ()

dq18 = pd.read_csv(pathq + "Questions Jun 2023.csv", low_memory=False)
print ("Jun 2023")
dq18 = DataClean(dq18, dl2refwoList, dquesList )
print ()

dq19 = pd.read_csv(pathq + "Questions Jul 2023.csv", low_memory=False)
print ("Jul 2023")
dq19 = DataClean(dq19, dl2refwoList, dquesList )
print ()

dq20 = pd.read_csv(pathq + "Questions Aug 2023.csv", low_memory=False)
print ("Aug 2023")
dq20 = DataClean(dq20, dl2refwoList, dquesList )
print ()


frames = [dq1, dq2, dq3, dq4, dq5,dq6, dq7, dq8, dq9, dq10, dq11, dq12, dq13, dq14, dq15, dq16, dq17, dq18, dq19, dq20 ]

dq = pd.concat(frames)
dq = dq.reset_index(drop=True)

print (dq.shape)

dq = dq[["wonum1", "Question Location", "result"]]

print ("DONE")



In [ ]:
#dq.to_csv(path + "test.csv", index=False)

Subset using Work Order Numbers

In [ ]:
print (str(datetime.datetime.today()) )

dq["wonum1"] = dq["wonum1"].astype(str).map(str.strip)
dl2['refwo'] = dl2['refwo'].astype(str).map(str.strip)

wolist = list(set(dq["wonum1"].tolist()  ))

print (len(dl2))
dl3 = dl2[dl2['refwo'].isin(wolist)].reset_index(drop=True)
print (len(dl3))

In [ ]:
transtype = set(dl3["transtype"].tolist() )
transtype

In [ ]:
dl3.to_csv(path + "test.csv", index=False)

### Insert Qustions and Results into Labor Transactions

In [ ]:
print (str(datetime.datetime.today()) )


#desc = sorted(list(set(dq["description3"].tolist() ) ))
#desc.remove('Adjacent Apartment 1')
#desc.remove('Adjacent Apartment 2')
#desc.remove('Adjacent Apartment 3')
#desc.remove('Adjacent Apartment 4')


#desc = ["Was Resident Educational Literature issued","Frass Removal", "HEPA Vacuum", 
#       "Fill holes with excluder mesh or similar product","Sealing", "Escutcheon Plate Installation" ]

desc = list(set(dq["Question Location"].tolist() ))

dq2 = dq.copy()

dq2["wonum1"] = dq2["wonum1"].astype(str).map(str.strip)
dl3['refwo'] = dl3['refwo'].astype(str).map(str.strip)


for d in desc:
    #print (d)
    dq3 = dq2[dq2["Question Location"] == d]
    print (d)
    print (len(dq3) )
    #print (dq3)
    
    dq3Dic = dq3.set_index('wonum1')['result'].to_dict()

    dl3[d] = dl3["refwo"].map(dq3Dic)



### Calculate Time

In [ ]:
print (str(datetime.datetime.today()) )
print ("Calculate time")

dl3['startdatetime'] = pd.to_datetime(dl3['startdatetime'])

dl3['finishdatetime'] = pd.to_datetime(dl3['finishdatetime'])


#df2['Hours'] = (df2["actfinish"] - df2["actstart"]).dt.days

pd.set_option('chained_assignment', None)

dl3 = dl3.reset_index(drop=True)

dl3['Seconds'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds
dl3['Hours'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds / 3600.0
dl3['Minutes'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds / 60.0
dl3['Days'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.days

dl3["Hours"] = dl3["Hours"].astype(float)
dl3["Hours"] = dl3["Hours"].round(2)

dl3["Minutes"] = dl3["Minutes"].astype(float)
dl3["Minutes"] = dl3["Minutes"].round(2)
print ("DONE")

### Merge Building and Development Data

In [ ]:
import pandas as pd

print (str(datetime.datetime.today()) )

dl3['location'] = dl3['location'].astype(str)

dl3["TDS"] = dl3['location'].str.split(".").str[0]

print ("Import nycha_org_dim")

import time
start = time.time()

import datetime
print (str(datetime.datetime.today()).split()[0] )

#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


nycha_org_query = """

select *
       
from maximo.nycha_org_dim nychaorg


"""

dnycha = pd.read_sql_query(nycha_org_query, engine)

subset = ['tds_num', 'borough_name', 'development_name']

dnycha= dnycha[subset]


tds = dnycha.tds_num.tolist()

tdslist = []

for c in tds:
    c = str(c)
    
    #print (c)
    
    if len(c) == 1:
        c = "00" + c
        tdslist.append(c)
    elif len(c) == 2:
        c = "0" + c
        tdslist.append(c)
    else:
        tdslist.append(c)


print (len(tdslist))
print (len(dnycha))

qn = pd.DataFrame(tdslist)

qn = qn.rename(columns={0: 'TDS'})

dnycha = pd.concat([dnycha, qn], axis=1)

print ("Merge with labor transactions")
print (len(dl3))
dl3 = pd.merge(dl3, dnycha, how='left', on=['TDS'])
print (len(dl3))


### Insert Address

In [ ]:
print (str(datetime.datetime.today()).split()[0] )

path2 = ""

db = pd.read_csv(path2 + "zzbuildings.csv", low_memory=False)

db["BUILDING"] = db["BUILDING"].astype(str)
db["BLDGNO"] = db["BLDGNO"].astype(str)

db["TDS"] = db["BUILDING"].str.split(".").str[0]

#convert 1 to 01
for i in range(0, len(db)):
    a = db["BLDGNO"][i]
    if len(a) == 1:
        db["BLDGNO"][i] = "0" + db["BLDGNO"][i]
        
    b = db["TDS"][i]
    if len(b) == 1:
        db["TDS"][i] = "00" + db["TDS"][i]
    if len(b) == 2:
        db["TDS"][i] = "0" + db["TDS"][i] 
        


db["Building"] = db["TDS"] + "." + db["BLDGNO"]

sitid = set(db["SITEID"].tolist() )
print(sitid)

db["SITEID"] = db["SITEID"].str.replace('BK', 'BROOKYN')
db["SITEID"] = db["SITEID"].str.replace('BX', 'BRONX')
db["SITEID"] = db["SITEID"].str.replace('QS', 'QUEENS/STATEN ISLAND')
db["SITEID"] = db["SITEID"].str.replace('MN', 'MANHATTAN')

db["ADDRESS"] = db["ADDRESS"] + ", " + db["SITEID"]


dl3["Building No"] = dl3['location'].str.split(".").str[1]

dl3["TDS"] = dl3["TDS"].astype(str)
dl3["Building"] = dl3["TDS"] + "." + dl3["Building No"]


#dictionary to insert data
dbDic = db.set_index('Building')['ADDRESS'].to_dict()

dl3["Building Address"] = dl3["Building"].map(dbDic)
dl3["Building Address"] = dl3["Building Address"].astype(str).map(str.strip)

print ("DONE")

### Insert Lat/Lon

In [ ]:
print (str(datetime.datetime.today()).split()[0] )

path2 = ""

dgis = pd.read_csv(path2 + "nycha_loc_gps.csv", low_memory=False)

dgis["TDS_NUM"] = dgis["TDS_NUM"].astype(str)
dgis["BLDG_NUM"] = dgis["BLDG_NUM"].astype(str)

for i in range(0, len(dgis)):
    a = dgis["BLDG_NUM"][i]
    if len(a) == 1:
        dgis["BLDG_NUM"][i] = "0" + dgis["BLDG_NUM"][i]
        
    b = dgis["TDS_NUM"][i]
    if len(b) == 1:
        dgis["TDS_NUM"][i] = "00" + dgis["TDS_NUM"][i]
    if len(b) == 2:
        dgis["TDS_NUM"][i] = "0" + dgis["TDS_NUM"][i]            

dgis["Building"] = dgis["TDS_NUM"] + "." + dgis["BLDG_NUM"]

lat = dgis.set_index('Building')['LATITUDE'].to_dict()
dl3["lat"] = dl3["Building"].map(lat)

lon = dgis.set_index('Building')['LONGITUDE'].to_dict()
dl3["lon"] = dl3["Building"].map(lon)

print ("DONE")


### Flag Work orders < 10 Minutes > 30 Minutes

In [ ]:
print (str(datetime.datetime.today()).split()[0] )

minutes = dl3["Minutes"].tolist()


mintues30flag = []
mintues10flag = []
mintues20flag = []
#greater10minutesflag = []


for i in range(0, len(minutes)):
    if minutes[i] <= 30:
        mintues30flag.append("Yes")
    else:
        mintues30flag.append("No")
        
        
    if minutes[i] <= 20:
        mintues20flag.append("Yes")
    else:
        mintues20flag.append("No")
        
        
    if minutes[i] <= 10:
        mintues10flag.append("Yes")
    else:
        mintues10flag.append("No")
        
    
        
        
    #if minutes[i] > 120:
    #    mintues120flag.append(1)
    #else:
    #    mintues120flag.append(0)
        
    #if minutes[i] > 10:
    #    greater10minutesflag.append("Yes")
    #else:
    #    greater10minutesflag.append("No")
        
        
        
print (len(mintues30flag))
print (len(mintues20flag))
print (len(mintues10flag))


#print (len(mintues120flag))
#print (len(greater10minutesflag))
print (len(dl3))

dl3["<=30 min"] = mintues30flag
dl3["<=20 min"] = mintues20flag
dl3["<=10 min"] = mintues10flag

#dl3[">120 min"] = mintues120flag
#dl3[">10 min"] = greater10minutesflag

print ("DONE")

### Create Apartment String

In [ ]:
dl3 = dl3.reset_index(drop=True)

#line = "071.04.008.F01.01A.KIT01"
#updated_line = '.'.join(line.split('.')[:-1])
print (str(datetime.datetime.today()).split()[0] )


updated_line = []

for i in range(0, len(dl3)):
    line = dl3["location"][i]
    linesplit = line.split(".")
    
    if len(linesplit) == 6:
        a = '.'.join(line.split('.')[:-1])
        #print (a)
        updated_line.append(a)
    else:
        #print (line)
        updated_line.append(line)

print (len(dl3))
print (len(updated_line))

dl3["Apartment"] = updated_line

In [ ]:
"""
for c in desc:
    d = set(dl3[c].tolist())
    print (c)
    print (d)
""" 

### Calculate occurence of questions

In [ ]:
#dl3["Percent No"] = ""
print (str(datetime.datetime.today()).split()[0] )


dl3['HAVE YOU VERIFIED THE 5 ALIVE IN AREAS YOU WERE ABLE TO INSPECT. 5 ALIVE VERIFICATION'] = dl3['HAVE YOU VERIFIED THE 5 ALIVE IN AREAS YOU WERE ABLE TO INSPECT. 5 ALIVE VERIFICATION'].str.replace('Y', 'S')
dl3['HAVE YOU VERIFIED THE 5 ALIVE IN AREAS YOU WERE ABLE TO INSPECT. 5 ALIVE VERIFICATION'] = dl3['HAVE YOU VERIFIED THE 5 ALIVE IN AREAS YOU WERE ABLE TO INSPECT. 5 ALIVE VERIFICATION'].str.replace('N', 'U')


for c in desc:
    dl3[c] = dl3[c].astype(str)

percent_u = []
percent_s = []
percent_nan = []
percent_cat = []


for i in range(0, len(dl3)):
#for i in range(0, 2):

    count = []
    for c in desc:
        d = dl3[c][i]
        count.append(d)
    
    #d = dl3["Frass Removal"][i]
    #count.append(d)
    
    #d = dl3["HEPA Vacuum"][i]
    #count.append(d)
    
    #d = dl3["Fill holes with excluder mesh or similar product"][i]
    #count.append(d)
    
    #d = dl3["Sealing"][i]
    #count.append(d)
    
    #d = dl3["Escutcheon Plate Installation"][i]
    #count.append(d)
   
    u = (count.count('U')/len(desc))
    u = round(u, 3)
    s = (count.count('S')/len(desc))
    s = round(s, 3)
    nan = (count.count('nan')/len(desc))
    nan = round(nan, 3)
    cat = (count.count('CAT')/len(desc))
    cat = round(cat, 3)

 
    percent_u.append(u)
    percent_s.append(s)
    percent_nan.append(nan)
    percent_cat.append(cat)
    

print (len(dl3))   
print (len(percent_u))   
print (len(percent_s))   
print (len(percent_nan))   
print (len(percent_cat))   

 
dl3["Percent U"] = percent_u
dl3["Percent S"] = percent_s
dl3["Percent nan"] = percent_nan
dl3["Percent CAT"] = percent_cat
print ("DONE")


In [ ]:
dl3.head(3)

In [ ]:
dl3.to_csv(path + "test.csv", index=False)

### Convert Transaction Type to Dummies

In [ ]:


print (str(datetime.datetime.today()).split()[0] )

print ("Convert transactions to dummies")
transaction_dummies = pd.get_dummies(dl3["transtype"])


#transaction_dummies["WORK"] = transaction_dummies["WORK"] + transaction_dummies["WORKWITHSEQ"]

#transaction_dummies = transaction_dummies.drop('WORKWITHSEQ', 1)

transaction_dummies.to_csv ( path + "test.csv", index=False)

print (len(transaction_dummies))
print (len(dl3))

dl4 = pd.concat([dl3, transaction_dummies], axis=1)

print (len(dl4))


dl4["INSPECTION"] = dl4["INSPECTION"].astype(str)
dl4["INSPECTION"] = dl4["INSPECTION"].str.replace('True', '1')
dl4["INSPECTION"] = dl4["INSPECTION"].str.replace('False', '0')

dl4["NOBLDGACCESS"] = dl4["NOBLDGACCESS"].astype(str)
dl4["NOBLDGACCESS"] = dl4["NOBLDGACCESS"].str.replace('True', '1')
dl4["NOBLDGACCESS"] = dl4["NOBLDGACCESS"].str.replace('False', '0')

dl4["RESNOTHOME"] = dl4["RESNOTHOME"].astype(str)
dl4["RESNOTHOME"] = dl4["RESNOTHOME"].str.replace('True', '1')
dl4["RESNOTHOME"] = dl4["RESNOTHOME"].str.replace('False', '0')

dl4["RESREFUSED"] = dl4["RESREFUSED"].astype(str)
dl4["RESREFUSED"] = dl4["RESREFUSED"].str.replace('True', '1')
dl4["RESREFUSED"] = dl4["RESREFUSED"].str.replace('False', '0')

print ("DONE")



In [ ]:
dl4

### Remove Non-NYCHA Employees

In [ ]:
"""

print (str(datetime.datetime.today()).split()[0] )


print ("remove non nycha employees")

dn = pd.read_csv(path + "7A Exterminators Group_Validation_Records as of 03-30-2023.csv")

dn["Badge #"] = dn["Badge #"].astype(str).map(str.strip)

badge = dn["Badge #"].tolist()


for i in range(len(badge)):
    #print (len(badge[i]))
    
    if len(badge[i]) == 2:
        badge[i] = "0000" + badge[i]
        
    if len(badge[i]) == 3:
        badge[i] = "000" + badge[i]
        
    if len(badge[i]) == 4:
        badge[i] = "00" + badge[i]
        
    if len(badge[i]) == 5:
        badge[i] = "0" + badge[i]
        
dn["Badge #"] = badge

dn["Employee Type"] = "NYCHA Employee"

dnDic = dn.set_index('Badge #')['Employee Type'].to_dict()


dl4["laborcode"] = dl4["laborcode"].astype(str).map(str.strip)

dl4["Employee Type"] = dl4["laborcode"].map(dnDic)

dl4["Employee Type"] = dl4["Employee Type"].fillna("Other")

print ("len before drop", len(dl4))

dl4 = dl4[dl4["Employee Type"] == "NYCHA Employee"]

print ("len after drop", len(dl4))

print ("DONE")


dl4 = dl4.reset_index(drop=True)
"""

### Change Field Names

In [ ]:
#dl4 = dl3.copy()

dl4["Finish Date"] = dl4["finishdatetime"].apply(lambda x: x.date())
dl4["Start Date"] = dl4["startdatetime"].apply(lambda x: x.date())
dl4 = dl4.rename(columns={'problemcode': 'Problem Code'})
dl4 = dl4.rename(columns={'failurecode': 'Failure Code'})
dl4 = dl4.rename(columns={'ownergroup': 'Owner Group'})
dl4 = dl4.rename(columns={'transtype': 'Transaction Type'})
dl4 = dl4.rename(columns={'status': 'Status'})
dl4 = dl4.rename(columns={'displayname': 'Name'})
dl4 = dl4.rename(columns={'department': 'Department'})
dl4 = dl4.rename(columns={'title': 'Title'})
dl4 = dl4.rename(columns={'zzhrtitle': 'HR Title'})
dl4 = dl4.rename(columns={'zzcraft': 'Craft'})
dl4 = dl4.rename(columns={'Enterdate': 'Enter Date'})


dl4["Count"] = 1

### Calculate Mode

In [ ]:
#transtype = list(set(dl4["Transaction Type"].tolist() ) )
#transtype
#transtype = list(set(dl4["Transaction Type 2"].tolist() ) )
#transtype


In [ ]:
"""
print (str(datetime.datetime.today()).split()[0] )

#dl5 = dl4["Name", "Transaction Type", "Minutes"]

dl4["Transaction Type 2"] = dl4["Transaction Type"]
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('WORKWITHSEQ', 'WORK')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('NOBLDGACCESS', 'NO ACCESS')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('RESNOTHOME', 'NO ACCESS')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('RESREFUSED', 'NO ACCESS')

transtype = list(set(dl4["Transaction Type 2"].tolist() ) )

print ("WORK")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "WORK"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "WORK" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn1 = qn.copy()



print ("NO ACCESS")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "NO ACCESS"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "NO ACCESS" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn2 = qn.copy()



print ("UNFOUNDED")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "NO ACCESS"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "UNFOUNDED" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn3 = qn.copy()


frames = [qn1, qn2, qn3]
result = pd.concat(frames).reset_index(drop=True)

print ("DONE")

"""

### Calculate Number of People work On a Job

In [ ]:
"""
print (str(datetime.datetime.today()).split()[0] )

print ("Calculate number of people")

dol = dl4[["location", "Finish Date", "Name", "startdatetime", "finishdatetime", "Transaction Type" ] ]

dol = dol.sort_values(by = 'location', ascending=True).reset_index(drop=True)

dol["Finish Date"] = dol["Finish Date"].astype(str).map(str.strip)
dol["LocationDate"] = dol["location"] + " " + dol["Finish Date"]

locations = sorted(set(dol["location"].tolist() ))

#print (len(dol))
#dol = dol.drop_duplicates(["location", "Finish Date", "Name"]).reset_index(drop=True)
#print (len(dol))

numberList = []
nameList = []

for i in range(0, len(locations)):
#for i in range(0, 1000):

    l = locations[i]
    
    #print (l)
    #print (l)
    dol2 = dol[dol["location"] == l]
    
    dol2 = dol2.sort_values(by = 'Finish Date', ascending=True).reset_index(drop=True)
    
    #print (dol2)
    #print("******************")
    
    
    d = sorted(set(dol2["Finish Date"].tolist() ))
    
    #print (len(d))
    
    for j in range(0, len(d)):
        dol3 = dol2[dol2["Finish Date"] == d[j]].reset_index(drop=True)
        dol3["Name"] = dol3["Name"].astype(str)
        names = sorted(set(dol3["Name"].tolist() ))
        
        if (len(dol3) > 1 and len(names)>1):
            #print (dol3)
            #print("******************")
            #numberDic = {dol3["LocationDate"][0]}
            #numberDic[dol3["LocationDate"][0]] = len(names)
            numberList.append([dol3["LocationDate"][0], len(names)])
            nameList.append([dol3["LocationDate"][0], names])


    #print(dol2)
    #print("******************")
    
print (len(numberList))
print (len(nameList))
print ("DONE")

"""

In [ ]:
path

Create Dictionaries and Combine with Labor Transactions

In [ ]:
"""
print ("Insert number on the job")
numberDict = {}
for l2 in numberList:
    numberDict[l2[0]] = l2[1]
    
nameDict = {}
for l2 in nameList:
    nameDict[l2[0]] = l2[1]
    
dl4["Finish Date"] = dl4["Finish Date"].astype(str).map(str.strip)
dl4["LocationDate"] = dl4["location"] + " " + dl4["Finish Date"]

dl4["Number on Job"] = dl4["LocationDate"].map(numberDict)
dl4["Employees on Job > 1"] = dl4["LocationDate"].map(nameDict)

#dl4 = dl4.drop('Employees on Job', 1)

dl4["Number on Job"] = dl4["Number on Job"].fillna(1)

print ("DONE")
"""

In [ ]:
#cols = dl4.columns
#cols = sorted(cols)
#cols

In [ ]:
#dol.to_csv(path + "test_num_people.csv", index=False)

### Calculate Work Time per Day

In [ ]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Radius of the Earth in miles
    earth_radius = 3959.0  # Mean radius in miles
    
    # Calculate the differences in coordinates
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c
    
    return distance

# Example coordinates: New York City and Los Angeles
lat1= 40.713196
lon1= -73.98156857

lat2= 40.79802487
lon2= -73.94695356

distance = haversine_distance(lat1, lon1, lat2, lon2)
print(f"Distance between points: {distance:.2f} miles")

In [ ]:
dl4["Transaction Type"]

In [ ]:
pathout = ""

print (len(dl4))
dl4["Name"] = dl4["Name"].astype(str)
dl4 = dl4[dl4["Name"] != "None"]
print (len(dl4))

dl4["Start Date"] = dl4["startdatetime"].apply(lambda x: x.date())
dl4["Start Time"] = dl4["startdatetime"]
dl4["Finish Time"] = dl4["finishdatetime"]


#dl4["Development"] = dl4['location'].str.split(".").str[0]
#dl4["Development"] = dl4["Development"].map(str.strip)


dl5 = dl4[["Name", "Start Date", "Finish Date", "Start Time", "Finish Time", "Minutes", "development_name", "lat", "lon",
          "Transaction Type"]]

dl5 = dl5[dl5["Transaction Type"] == "INSPECTION"].reset_index(drop=True)

dl5 = dl5.sort_values(by = 'Start Date', ascending=True).reset_index(drop=True)

dl5["Start Date"] = dl5["Start Date"].astype(str).map(str.strip)
dl5["Finish Date"] = dl5["Finish Date"].astype(str).map(str.strip)

dates = sorted(list(set(dl5["Start Date"].tolist() )))

len(dates)

dt = pd.DataFrame(columns=('Name', 'Start Date', 'Finish Date', 'Day Start Time', 'Day End Time', 
                           'Number of Jobs', 'Minutes Worked', 'Developments', 'Distance Traveled (mi)'))

all_data = pd.DataFrame()

for i in range(0, len(dates)):
#for i in range(0, 1):

    #print(i)
    dl6 = dl5[dl5["Start Date"] == dates[i]]
    dl6 = dl6[dl6["Finish Date"] == dates[i]]
    dl6 = dl6.reset_index(drop=True)
    
    names = list(set(dl6["Name"].tolist() ))
    
    for j in range(0, len(names)):
    #for j in range(0, 1):
        
        distlist = []
    
        dl7 = dl6[dl6["Name"] == names[j]]
        
        dl7 = dl7.sort_values(by = 'Start Time', ascending=True).reset_index(drop=True)
        
        developments = sorted(list(set(dl7["development_name"].tolist() )) )
        lat = dl7["lat"].tolist()
        lon = dl7["lon"].tolist()
        
        for k in range(0, len(lat)-1):
            distance = haversine_distance(lat[k], lon[k], lat[k+1], lon[k+1])
            distlist.append(distance)
            #print (len(distlist))
            #print (distance)
            #print ()

        
        
        totalMinWorked = dl7["Minutes"].sum()
        
        starttime = sorted(dl7["Start Time"].tolist() )
        endtime = sorted(dl7["Finish Time"].tolist(),  reverse=True )


        start = starttime[0]
        end = endtime[0]
        
        #print (start)
        #print (end)
        
        sumdistlist = sum(distlist)
        
        dt.loc[j] = [names[j],dates[i],dates[i], start, end, len(dl7), totalMinWorked, str(developments), sumdistlist]
        
        fileout = "work_time" + str(i) + ".csv"

        dt.to_csv(pathout + fileout, index=False)


    #all_data = all_data.append(dt,ignore_index=False).reset_index(drop=True)

    

#all_data = all_data.sort_values(by = 'Start Date', ascending=True).reset_index(drop=True)


#all_data['Hours Worked'] = (all_data['Day End Time'] - all_data['Day Start Time']).dt.total_seconds()/3600
#all_data['Minutes Worked'] = (all_data['Day End Time'] - all_data['Day Start Time']).dt.total_seconds()/60

#all_data['Day Start Time'] = all_data['Day Start Time'].apply( lambda d : d.time() )

#all_data['Day End Time'] = all_data['Day End Time'].apply( lambda d : d.time() )

#all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
#all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]


In [ ]:
dl4.to_csv(path + "test.csv", index=False)

In [ ]:
import pandas as pd
import glob
import os

print ("Read csv files back into a data frame")

# Get a list of all CSV file paths in a directory
csv_files = glob.glob( '*.csv')

# Create an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    dt = pd.read_csv(file)
    dataframes.append(dt)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

all_data = final_df.copy()

pathread = ''

# Get a list of all CSV file paths in the directory
csv_files = glob.glob(os.path.join(pathread, '*.csv'))

# Loop through each CSV file and delete it
for file_path in csv_files:
    os.remove(file_path)

print("CSV files deleted successfully.")

print ("Done")




Data Cleaning

In [ ]:
all_data["Hours Worked"] = all_data["Minutes Worked"]/60.0
all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]


"""
all_data['Hours Worked'] = ""
#all_data['Minutes Worked'] = ""

for i in range(0, len(all_data)):
    d = all_data["Day End Time"][i] - all_data["Day Start Time"][i]

    diff = d.seconds

    diff = int(diff)
    
    all_data['Hours Worked'][i] = diff/3600.0
    #all_data['Minutes Worked'][i] = diff/60.0


    #print (diff)
    
all_data = all_data.sort_values(by = 'Hours Worked', ascending=False).reset_index(drop=True)

all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]
"""

In [ ]:
all_data2 = all_data.drop_duplicates(subset=['Name', 'Day Start Time', 'Day End Time', 'Number of Jobs']).reset_index(drop=True)
print (len(all_data))
print (len(all_data2))


Outside Normal Work Hours

In [ ]:
#all_data2["Day End Time"][0].hour

all_data2["Day Start Time"] = pd.to_datetime(all_data2["Day Start Time"])
all_data2["Day End Time"] = pd.to_datetime(all_data2["Day End Time"])

all_data2["Outside Normal Work Hours"] = "No"

for i in range(0, len(all_data2)):
    if (all_data2["Day Start Time"][i].hour <= 7 or all_data2["Day End Time"][i].hour >= 21):
        all_data2["Outside Normal Work Hours"][i] = "Yes"

In [ ]:
all_data2["Day Start Time"][0]

### Mode: Start and End Time

In [ ]:
all_data2["Day Start Time"][0].hour
all_data2["Start Hour"] = ""
all_data2["End Hour"] = ""

for i in range(0, len(all_data2)):
    all_data2["Start Hour"][i] =  all_data2["Day Start Time"][i].hour
    all_data2["End Hour"][i] =  all_data2["Day End Time"][i].hour


In [ ]:
names = sorted(list(set(all_data2["Name"].tolist()  )))

qnhours = pd.DataFrame(columns=('Emplpoyee Name', 'Mode Start Time', 'Count of Start Mode', 
                                'Mode End Time', 'Count of End Mode', 'Total Number of Days Worked'))


#dl4["Minutes"] = dl4["Minutes"].astype(int)
all_data2["Start Hour Cat"] = all_data2["Start Hour"].astype(str)
all_data2["End Hour Cat"] = all_data2["End Hour"].astype(str)


for i in range(0, len(names)):
    n = names[i]
    
  
    all_data3 = all_data2[all_data2["Name"] == n].reset_index(drop=True)
    all_data3["Count"] = 1
    
    #start time mode
    all_data3G = all_data3[["Start Hour Cat", "Count"]]
    all_data3G = all_data3G.groupby(['Start Hour Cat']).sum()
    all_data3G = all_data3G.add_suffix('').reset_index()
    all_data3G = all_data3G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    starthourmode = all_data3G["Start Hour Cat"][0]
    starthourmodecount = all_data3G["Count"][0]
    
    #end time mode
    all_data3G = all_data3[["End Hour Cat", "Count"]]
    all_data3G = all_data3G.groupby(['End Hour Cat']).sum()
    all_data3G = all_data3G.add_suffix('').reset_index()
    all_data3G = all_data3G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    endhourmode = all_data3G["End Hour Cat"][0]
    endhourmodecount = all_data3G["Count"][0]
    
        
    if len(all_data3) > 0:
        qnhours.loc[i] = [n,starthourmode, starthourmodecount,endhourmode,endhourmodecount, len(all_data3)]

qnhours["Start Mode Relative Frequency"] = qnhours["Count of Start Mode"]/qnhours["Total Number of Days Worked"]
qnhours["Start Mode Relative Frequency"] = qnhours["Start Mode Relative Frequency"].astype(float)
qnhours["Start Mode Relative Frequency"] = qnhours["Start Mode Relative Frequency"].round(2)

qnhours["End Mode Relative Frequency"] = qnhours["Count of End Mode"]/qnhours["Total Number of Days Worked"]
qnhours["End Mode Relative Frequency"] = qnhours["End Mode Relative Frequency"].astype(float)
qnhours["End Mode Relative Frequency"] = qnhours["End Mode Relative Frequency"].round(2)

#qn1 = qn.copy()

In [ ]:
qnhours

### Calculate the Days of the Week

In [ ]:
# 0 = monday, 1=tuesday, 2=wednesday, 3=thursday, 4=friday, 5=saturday, 6=sunday
dl4["Day"] = dl4['Finish Time'].dt.dayofweek
dl4["Day"] = dl4["Day"].astype(str).map(str.strip)
dl4["Day"] = dl4["Day"].str.replace('0', 'Monday')
dl4["Day"] = dl4["Day"].str.replace('1', 'Tuesday')
dl4["Day"] = dl4["Day"].str.replace('2', 'Wednesday')
dl4["Day"] = dl4["Day"].str.replace('3', 'Thursday')
dl4["Day"] = dl4["Day"].str.replace('4', 'Friday')
dl4["Day"] = dl4["Day"].str.replace('5', 'Saturday')
dl4["Day"] = dl4["Day"].str.replace('6', 'Sunday')



In [ ]:
dl4 = dl4.reset_index(drop=True)


dl4['Finish Time 2'] = dl4["Finish Time"].apply( lambda d : d.time() )

dl4['Finish Time 2'] = dl4['Finish Time 2'].astype(str).map(str.strip)

dl4['Finish Time 2'] = dl4['Finish Time 2'].str.split(":").str[0]


dl4['Finish Time 2'] = dl4['Finish Time 2'].astype(int)

dl4["Work After 4pm"] = "No"

for i in range(0, len(dl4)):
    if dl4['Finish Time 2'][i] >= 16:
        dl4["Work After 4pm"][i] = "Yes"


#if dl4["Finish Time 2"][0] > pd.datetime('13:10:00'):
#    print ("hey")

### Create Dummes for Questions

In [ ]:


"""
a = dl4["Frass Removal"].tolist()
print (len(a))
print (len(dl4))

dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Education Literature"] = dummies 



a = dl4["Frass Removal"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Frass"] = dummies 


a = dl4["HEPA Vacuum"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["HEPA"] = dummies


a = dl4["Fill holes with excluder mesh or similar product"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Exluder mesh"] = dummies       


a = dl4["Sealing"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Seal"] = dummies       

a = dl4["Escutcheon Plate Installation"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)      
dl4["Escutcheon Plate"] = dummies       

print ("DONE")

"""

### Create Apartment and Public Space Category

In [ ]:
locationlist = dl4["location"].tolist()
workspace = []
for i in range(0, len(dl4)):
    a = locationlist[i]
    a = a.split(".")
    #print (a)
    
    if len(a) < 5:
        workspace.append("Public Space")
    else:
        workspace.append("Apartment")
        
print (len(dl4))
print (len(workspace))
    
dl4["Work Space Type"] =   workspace  

### Create Hour Field

In [ ]:
#dl4["Hour of Day"] = dl4["Finish Time"].hour

hours = dl4["Finish Time"].tolist()

hourofday = []

for i in range(0, len(hours)):
    #print (hours[i])
    
    a = hours[i].hour
    
    hourofday.append(a)
    
print (len(hourofday))
print (len(dl4))

dl4["Hour of Day"] = hourofday




### Create NYCHA employee field

In [ ]:
"""

dp = pd.read_csv(path + "Pest Management Routes as of 1 31 23 CORRECTED.csv")
dp["Name"] = dp["Name"].astype(str).map(str.upper).map(str.strip)
dp["Last Name"] = dp["Name"] .str.split(",").str[0].map(str.strip)
dp["First Name"] = dp["Name"] .str.split(",").str[1].map(str.strip)

dp["Last Name"] = dp["Last Name"].map(str.strip)
dp["First Name"] = dp["First Name"].map(str.strip)

dp["Full Name"] = dp["First Name"] + " " + dp["Last Name"]

dp["NYCHA Employee"] = "NYCHA Employee (1_31_23)"

dp["Full Name"] = dp["Full Name"].map(str.strip)

dp["Full Name"] = dp["Full Name"].str.replace('AGATHA DESVIGNES-MCKAIN', 'AGATHA DESVIGNES MCKAIN')
dp["Full Name"] = dp["Full Name"].str.replace('ALVEN LAKE', 'ALVIN LAKE')
dp["Full Name"] = dp["Full Name"].str.replace('BRIAN JEFFRIES', 'BRIAN JEFFERIES')
dp["Full Name"] = dp["Full Name"].str.replace('CELESTINE R PAYNE', 'CELESTINE PAYNE')
dp["Full Name"] = dp["Full Name"].str.replace('DIEDRA EPHRAIM', 'DEIDRA EPHRAIM')
dp["Full Name"] = dp["Full Name"].str.replace('DIOGENES UMANZAR', 'DIOGENES UMANZOR')
dp["Full Name"] = dp["Full Name"].str.replace('GEOVANI DORSEY', 'GEOVANNIE DORSEY')
dp["Full Name"] = dp["Full Name"].str.replace('HERIBERTO CANTRES', 'HERIBERTO CANTES')
dp["Full Name"] = dp["Full Name"].str.replace('JEMEEL TUCKER', 'JAMEEL TUCKER')
dp["Full Name"] = dp["Full Name"].str.replace('JOSEPH UPSUR', 'JOSEPH UPSHUR')
dp["Full Name"] = dp["Full Name"].str.replace('LAYR DORZIN', 'LAYR DORZIN, JR')
dp["Full Name"] = dp["Full Name"].str.replace('LENAIR DANIELS', 'LENAIR DANIEL')
dp["Full Name"] = dp["Full Name"].str.replace('LORRAINE LOWERY- CLARK', 'LORRAINE LOWERY-CLARK')
dp["Full Name"] = dp["Full Name"].str.replace('MATTEW MARTINEZ', 'MATTHEW MARTINEZ')
dp["Full Name"] = dp["Full Name"].str.replace('NUDUKA OKORO', 'NDUKA OKORO')
dp["Full Name"] = dp["Full Name"].str.replace('OLUYINKA ODUMNBAKU', 'OLUYINKA ODUNMBAKU')
dp["Full Name"] = dp["Full Name"].str.replace('SALVATORE CASTELLENO', 'SALVATORE CASTELLANO')
dp["Full Name"] = dp["Full Name"].str.replace('SHERIF RASHID', 'SHAREEF RASHID')
dp["Full Name"] = dp["Full Name"].str.replace('SHANICE CALLWOOD', 'SHAUNICE CALLWOOD')
dp["Full Name"] = dp["Full Name"].str.replace('SOLOMON SOOKOO', 'SOLOMAN SOOKOO')

dp["Full Name"] = dp["Full Name"].str.replace('VICAS FRANCIS', 'VICKAS FRANCIS')


dpDic = dp.set_index('Full Name')['NYCHA Employee'].to_dict()

dpList = dp["Full Name"].tolist()


"""




In [ ]:
"""

dln = dl4[["Name"]]
dln["Name"] = dln["Name"].astype(str).map(str.upper).map(str.strip)
dln = dln.drop_duplicates(['Name']).reset_index(drop=True)

dln["NYCHA Employee"] = dln["Name"].map(dpDic)

dln = dln.sort_values(by = 'Name', ascending=True).reset_index(drop=True)

dlnList = dln["Name"].tolist()

#dln.to_csv(path + "test_names.csv", index=False)

from fuzzywuzzy import fuzz

from operator import itemgetter


def fuzzy_match(string1, string2):
    
    #Compares two strings using fuzzy matching and returns a score between 0 and 100.

    return fuzz.ratio(string1, string2)

dln["NYCHA Employee"] = dln["NYCHA Employee"].astype(str)
dln["Fuzzy Name Closest Match"] = ""
dln["Fuzzy Score"] = ""

for i in range(0, len(dln)):
    if dln["NYCHA Employee"][i]=='nan' :
        n = dln["Name"][i]
        #print (n)
        
        namematch = []

        for j in range(0, len(dpList)):
            #print (dpList[j], fuzzy_match(n, dpList[j]))
            namematch.append([dpList[j], fuzzy_match(n, dpList[j])])
            
            namematch = sorted(namematch, key=itemgetter(1), reverse=True)

        dln["Fuzzy Name Closest Match"][i] = namematch[0][0]
        dln["Fuzzy Score"][i] = namematch[0][1]


"""

In [ ]:
#dln.to_csv(path + "test_names.csv", index=False)

### Output

In [ ]:
path

In [ ]:
all_data2.to_csv(path + "Work Time Analytics.csv", index=False)

In [ ]:
qnhours.to_csv(path + "Start and End Time Modes.csv", index=False)

In [ ]:
#result.to_csv(path + "Time Mode and Employee Names.csv", index=False)

In [ ]:
dl4.to_csv(path + "Apartment Inspection Labor Transactions.csv", index=False)

In [ ]:
#dl4.to_csv(path + "test.csv", index=False)

In [ ]:
numinsp = list(set(dl4["Name"].tolist() ))
print (len(numinsp))

# Calculate Outliers

### Minutes worked per inspection

In [ ]:
all_data3 = all_data2.copy()

all_data3 = all_data3[["Name", "Number of Jobs", "Minutes Worked"]]

all_data3 = all_data3.sort_values(by = 'Minutes Worked', ascending=False).reset_index(drop=True)

all_data3G = all_data3.groupby(['Name']).sum()

all_data3G = all_data3G.add_suffix('').reset_index()



all_data3G["Avg Min Inspection"] = all_data3G["Minutes Worked"]/all_data3G["Number of Jobs"]

#all_data3G = all_data3G.sort_values(by = 'Avg Min Inspection', ascending=True).reset_index(drop=True)


q1 = all_data3G['Number of Jobs'].quantile(0.25)  # First quartile (25th percentile)
q2 = all_data3G['Number of Jobs'].quantile(0.50)  # First quartile (25th percentile)
q3 = all_data3G['Number of Jobs'].quantile(0.75)  # First quartile (25th percentile)
q4 = all_data3G['Number of Jobs'].quantile(1.00)  # First quartile (25th percentile)

print ("q1", q1)
print ("q2", q2)
print ("q3", q3)
print ("q4", q4)

all_data3G = all_data3G.sort_values(by = 'Number of Jobs', ascending=False).reset_index(drop=True)

print (len(all_data3G))
all_data3G = all_data3G[all_data3G["Number of Jobs"] > 100]
print (len(all_data3G))

print ("Number of Inspections")
print ("Avg num inspections", all_data3G["Number of Jobs"].mean())
print ("Median num inspections", all_data3G["Number of Jobs"].median())
print ("Std num inspections", all_data3G["Number of Jobs"].std())

print()
print ("time per inspection")
print ("Avg min per inspection", all_data3G["Avg Min Inspection"].mean())
print ("Median per inspections", all_data3G["Avg Min Inspection"].median())
print ("Std dev min per inspection", all_data3G["Avg Min Inspection"].std())


all_data3G = all_data3G.sort_values(by = 'Avg Min Inspection', ascending=True).reset_index(drop=True)


In [ ]:
all_data3G[:50]

### Percent S

In [ ]:
dl5 = dl4[["Name", "Percent S"]]

dl5["Count"] = 1

#dl5G = dl5.groupby(['Name','Percent S', 'Count']).mean()

dl5G = (dl5.groupby('Name').agg({'Percent S':'mean', 'Count': 'sum'}).rename(columns={'Count':'Count'}) )


dl5G = dl5G.add_suffix('').reset_index()

print (len(dl5G))
dl5G = dl5G[dl5G["Count"] > 100]
print (len(dl5G))


dl5G = dl5G.sort_values(by = 'Percent S', ascending=False).reset_index(drop=True)


print()
print ("time per inspection")
print ("Avg percent S", dl5G["Percent S"].mean())
print ("Median percent S", dl5G["Percent S"].median())
print ("Std dev percent S", dl5G["Percent S"].std())


q1 = dl5G["Percent S"].quantile(0.25)  # First quartile (25th percentile)
q2 = dl5G["Percent S"].quantile(0.50)  # First quartile (25th percentile)
q3 = dl5G["Percent S"].quantile(0.75)  # First quartile (25th percentile)
q4 = dl5G["Percent S"].quantile(1.00)  # First quartile (25th percentile)

print ("q1", q1)
print ("q2", q2)
print ("q3", q3)
print ("q4", q4)


In [ ]:
print ("Percent S and Avg min per inspection")

dl5 = dl4.copy()

dl5["Count"] = 1

dl5["INSPECTION"] = dl5["INSPECTION"].astype(int)
dl5["RESNOTHOME"] = dl5["RESNOTHOME"].astype(int)

dl5 = dl5[["Name", "Minutes", "Percent S", "INSPECTION", "Count"]]

print (len(dl5))
dl5 = dl5[dl5["INSPECTION"] == 1]
print (len(dl5))

dl5G = (dl5.groupby('Name').agg({'Percent S':'mean', 'Count': 'sum', 'Minutes':'sum', 'INSPECTION':'sum'}).rename(columns={'Count':'Count'}) )

dl5G = dl5G.add_suffix('').reset_index()


print (len(dl5G))
dl5G = dl5G[dl5G["Count"] > 100]
print (len(dl5G))

dl5G["Avg min inspection"] = dl5G["Minutes"]/dl5G["Count"]

dl5G = dl5G.sort_values(by = 'Avg min inspection', ascending=True).reset_index(drop=True)


#dl6 = dl4.copy()
#dl6["Count"] = 1


In [ ]:
dl5G.to_csv(path + "Bad Actor List.csv", index=False)

In [ ]:
dl4.dtypes[50:]

In [ ]:
dl4["INSPECTION"]